In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('/content/medical appointment.csv')
print(df)

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])

In [ ]:
df['AppointmentDate'] = df['AppointmentDay'].dt.date
df['ScheduledDate'] = df['ScheduledDay'].dt.date

In [ ]:
df.info()

In [ ]:
print(df)

In [ ]:
del df['ScheduledDay']
del df['AppointmentDay']

In [ ]:
df.rename(columns={'Handcap': 'Handicap', 'Hipertension ': 'Hypertension ', 'No-show': 'No_show'}, inplace=True)

In [ ]:
df['No_show'] = df['No_show'].map({'Yes': 1, 'No': 0})

In [ ]:
print(df)

In [ ]:
df.info()

In [ ]:
df['No_show'].value_counts()

In [ ]:
df['Gender'].value_counts()

In [ ]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print("\n")

In [ ]:
df['Handicap'].value_counts()

In [ ]:
df=df[~df['Handicap'].isin([2,3,4])]

In [ ]:
df['Handicap'].value_counts()

In [ ]:
df.drop(df[(df['Age'] <= 1) | (df['Age'] > 100)].index, inplace=True)

In [ ]:
df['AppointmentDate'] = pd.to_datetime(df['AppointmentDate'])

In [ ]:
df['ScheduledDate'] = pd.to_datetime(df['ScheduledDate'])

In [ ]:
df['AppointmentWeek'] = df['AppointmentDate'].dt.day_name()

In [ ]:
df['AppointmentMonth'] = df['AppointmentDate'].dt.day

In [ ]:
df['waiting_days'] = (df.AppointmentDate - df.ScheduledDate) / pd.Timedelta(days=1)
df['waiting_days'] =df['waiting_days'].astype(int)

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df['waiting_days'])
plt.show()

In [ ]:
sns.stripplot(data=df, y='waiting_days')
plt.show()

In [ ]:
outliers = df[(df['waiting_days'] > 150)]

In [ ]:
df.drop(outliers.index, inplace=True)

In [ ]:
df.info()

In [ ]:
duplicate_rows = df[df.duplicated(subset=['PatientId'])]
len(duplicate_rows)

In [ ]:
df.drop(duplicate_rows.index, inplace=True)

In [ ]:
df.info()

In [ ]:
df.to_csv('cleaned_dataset.csv', index=False)

In [ ]:
df_new= pd.read_csv('/content/cleaned_dataset.csv')
print(df_new)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
df_new['Gender'] = le.fit_transform(df_new['Gender'])

In [ ]:
df_new['AppointmentWeek_number'] = le.fit_transform(df['AppointmentWeek'])

In [ ]:
print(df_new['Gender'])

In [ ]:
print(df_new['AppointmentWeek_number'])

In [ ]:
from sklearn.model_selection import train_test_split

X = df_new[['Age', 'Gender', 'AppointmentWeek_number', 'waiting_days', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received']]
y = df_new['No_show']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#!pip install -q category_encoders

In [ ]:
#from category_encoders import OneHotEncoder

In [ ]:
#encoder = OneHotEncoder(cols=['AppointmentWeek'], handle_unknown='ignore')

In [ ]:
#X_train = encoder.fit_transform(X_train)

In [ ]:
#X_test = encoder.transform(X_test)

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42)

In [ ]:
tree_model.fit(X_train, y_train)

In [ ]:
y_pred = tree_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#classification report (includes precision, recall, F1-score)
print(classification_report(y_test, y_pred))

In [ ]:
new_data = [[40, 0, 5, 20, 1, 1,1 , 1, 1]]

In [ ]:
prediction = tree_model.predict(new_data)
print(prediction)

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(12, 8))  # Set the figure size (adjust as needed)
plot_tree(tree_model, feature_names=X.columns, class_names=['No_show', 'show'], filled=True, rounded=True)
plt.show()

In [ ]:
plt.savefig('decision_tree.png')